# Task 4

In [83]:
# KNN cosine
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [84]:
df = pd.read_csv(
    '/content/movies.csv',
    on_bad_lines='skip',
    engine='python',
    encoding='utf-8'
)
movies_metadata = pd.read_csv(
    '/content/movies_metadata.csv',
    usecols=['id','imdb_id','title'],
    on_bad_lines='skip',
    engine='python',
    encoding='utf-8'
)

links = pd.read_csv(
    '/content/links.csv'
)

ratings_small = pd.read_csv(
    '/content/ratings_small.csv'
)

links = links.merge(ratings_small, on='movieId')
links.rename(columns={'tmdbId':'id'}, inplace=True)


movies_metadata = movies_metadata.drop([29503, 35587, 19730], errors='ignore')
movies_metadata['id'] = movies_metadata['id'].astype('int64')
movies_metadata = movies_metadata.merge(links, on='id')
print(len(movies_metadata))

print(movies_metadata.head())


73970
    id    imdb_id      title  movieId  imdbId  userId  rating   timestamp
0  862  tt0114709  Toy Story        1  114709       7   3.000   851866703
1  862  tt0114709  Toy Story        1  114709       9   4.000   938629179
2  862  tt0114709  Toy Story        1  114709      13   5.000  1331380058
3  862  tt0114709  Toy Story        1  114709      15   2.000   997938310
4  862  tt0114709  Toy Story        1  114709      19   3.000   855190091


In [85]:
df = movies_metadata
df = movies_metadata[['userId', 'movieId', 'rating', 'title']]
combine_movie_rating = df.dropna(axis = 0, subset = ['title'])

movie_ratingCount = (df.
                     groupby(by = ['title'])['rating'].
                     count().
                     reset_index().
                     rename(columns = {'rating': 'totalRatingCount'})
                     [['title', 'totalRatingCount']])

movie_ratingCount.head()
rating_with_totalRatingCount = combine_movie_rating.merge(movie_ratingCount, left_on = 'title', right_on = 'title', how = 'left')

rating_with_totalRatingCount.head()

,userId,movieId,rating,title,totalRatingCount
0,7,1,3.000,Toy Story,247
1,9,1,4.000,Toy Story,247
2,13,1,5.000,Toy Story,247
3,15,1,2.000,Toy Story,247
4,19,1,3.000,Toy Story,247


In [86]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(movie_ratingCount['totalRatingCount'].describe())

count   3937.000
mean      18.788
std       32.739
min        1.000
25%        2.000
50%        6.000
75%       20.000
max      341.000
Name: totalRatingCount, dtype: float64


In [87]:
popularity_threshold = 50
rating_popular_movie= rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
rating_popular_movie.head()

,userId,movieId,rating,title,totalRatingCount
0,7,1,3.000,Toy Story,247
1,9,1,4.000,Toy Story,247
2,13,1,5.000,Toy Story,247
3,15,1,2.000,Toy Story,247
4,19,1,3.000,Toy Story,247


In [88]:
movie_features_df = rating_popular_movie.pivot_table(index='title', columns='userId', values='rating').fillna(0)
movie_features_df.head(10)

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,3.000,0.000,0.000,0.000,0.000,0.000,0.000
12 Angry Men,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2001: A Space Odyssey,0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,...,0.000,0.000,4.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
A Beautiful Mind,0.000,0.000,0.000,0.000,4.500,0.000,0.000,3.500,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,4.000
A Bug's Life,0.000,0.000,0.000,0.000,3.500,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,4.000
A Christmas Story,0.000,0.000,0.000,5.000,0.000,0.000,0.000,3.500,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,5.000
A Clockwork Orange,0.000,0.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,3.500,3.000,0.000,0.000,0.000,0.000,0.000,3.000
A Close Shave,0.000,0.000,0.000,0.000,0.000,0.000,5.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,5.000,0.000,0.000,0.000,4.000
A Few Good Men,0.000,0.000,0.000,5.000,0.000,0.000,0.000,0.000,3.000,0.000,...,0.000,0.000,0.000,4.000,0.000,0.000,0.000,0.000,0.000,0.000


In [89]:
# Collaborative based Recommendation System using KNN

from scipy.sparse import csr_matrix

# converting the matrix into array matrix
movie_features_df_matrix = csr_matrix(movie_features_df.values)


from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')

model_knn.fit(movie_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [90]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.neighbors import (NeighborhoodComponentsAnalysis,KNeighborsClassifier)
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV
rating_popular_movie['rating']=rating_popular_movie['rating'].astype('int')
label_encoder= LabelEncoder()
rating_popular_movie['title']=label_encoder.fit_transform(rating_popular_movie['title'])
X,y=rating_popular_movie[['title']],rating_popular_movie['rating']
leaf_size = list(range(1,50))
n_neighbors = list(range(0,10))
p=[1,2,6,10]
#convert to dictionary
knn=KNeighborsClassifier(n_neighbors=10)
nca_pipe=Pipeline([('knn',knn)])
nca_pipe.fit(X,y)
#print(nca_pipe.score(X_test,y_test))
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
clf = GridSearchCV(knn, hyperparameters, cv=10)
best_model = clf.fit(X_train,y_train)

/tmp/ipython-input-2593618365.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating_popular_movie['rating']=rating_popular_movie['rating'].astype('int')
/tmp/ipython-input-2593618365.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating_popular_movie['title']=label_encoder.fit_transform(rating_popular_movie['title'])
/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
1960 fits failed out of a total of 19600.
The score on these train-test partitio

In [91]:
from sklearn.metrics import accuracy_score
#Predict testing set
y_pred = best_model.predict(X_test)
#Check performance using accuracy
print(accuracy_score(y_test, y_pred))

0.36585684430512017


In [92]:
print(movie_features_df_matrix)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 38180 stored elements and shape (390, 669)>
  Coords	Values
  (0, 12)	3.5
  (0, 14)	5.0
  (0, 44)	2.5
  (0, 55)	2.0
  (0, 60)	4.0
  (0, 72)	3.5
  (0, 73)	4.0
  (0, 94)	5.0
  (0, 97)	5.0
  (0, 110)	4.5
  (0, 124)	4.5
  (0, 127)	5.0
  (0, 133)	3.0
  (0, 148)	4.0
  (0, 149)	2.5
  (0, 167)	4.0
  (0, 185)	4.0
  (0, 205)	0.5
  (0, 210)	4.0
  (0, 211)	1.0
  (0, 236)	4.0
  (0, 237)	4.0
  (0, 253)	3.0
  (0, 266)	3.5
  (0, 287)	5.0
  :	:
  (389, 431)	4.0
  (389, 449)	4.0
  (389, 465)	3.5
  (389, 469)	4.0
  (389, 479)	5.0
  (389, 502)	3.5
  (389, 506)	4.0
  (389, 515)	3.0
  (389, 516)	4.0
  (389, 517)	3.5
  (389, 519)	4.0
  (389, 525)	4.0
  (389, 526)	4.0
  (389, 534)	4.0
  (389, 544)	5.0
  (389, 550)	4.0
  (389, 558)	4.5
  (389, 561)	5.0
  (389, 566)	1.0
  (389, 572)	4.0
  (389, 582)	5.0
  (389, 595)	3.5
  (389, 600)	5.0
  (389, 621)	3.0
  (389, 651)	4.5


In [93]:
def recommend_movie(movie_name):

    distances, indices = model_knn.kneighbors(movie_features_df.loc[movie_name,:].values.reshape(1, -1), n_neighbors = 6)

    for i in range(0, len(distances.flatten())):
        if i == 0:
            #print('Recommendations for {0}:\n'.format(movie_features_df.index[indices.flatten()[0]]))
            print('Recommendations for {0}:\n'.format(movie_name))
        else:
            print('{0}: {1}, with distance of {2}:'.format(i, movie_features_df.index[indices.flatten()[i]], distances.flatten()[i]))

In [94]:
recommend_movie('A Close Shave')

Recommendations for A Close Shave:

1: The Wrong Trousers, with distance of 0.3426323067106505:
2: This Is Spinal Tap, with distance of 0.6055093141588085:
3: Monty Python and the Holy Grail, with distance of 0.6101802075797369:
4: A Fish Called Wanda, with distance of 0.6147928467232779:
5: Life of Brian, with distance of 0.6189618572544828:


In [ ]:
# SAVE 
import pickle
with open("knn_model.pkl", "wb") as f:
    pickle.dump(model_knn, f)

with open("cosine_df.pkl", "wb") as f:
    pickle.dump(movie_features_df, f)